In [3]:
import ipywidgets as widgets
import numpy as np
from abtem import Probe
from abtem.visualize.interactive.artists import ImageArtist
from abtem.visualize.interactive.canvas import Canvas
from abtem.visualize.utils import domain_coloring
from abtem.visualize.interactive.utils import throttle

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

In [28]:
canvas1 = Canvas()
canvas2 = Canvas()

artist1 = ImageArtist(rgb=True)
canvas1.artists = {'image': artist1}

artist2 = ImageArtist(rgb=True)
canvas2.artists = {'image': artist2}

probe = Probe(energy=80e3, semiangle_cutoff=90, defocus=0,
              Cs=0e4, astigmatism=0, gpts=160, extent=20, rolloff=2)

defocus_slider = widgets.FloatSlider(description='Defocus', min=-1e2, max=1e2, value=0, step=1e1)
cs_slider = widgets.FloatSlider(description='Cs', min=-1e4, max=1e4, value=0, step=1e3)
c5_slider = widgets.FloatSlider(description='C5', min=-1e7, max=1e7, value=0, step=1e6)
astigmatism_slider = widgets.FloatSlider(description='Astigmatism', min=-200, max=200, value=0, step=10)
astigmatism_angle_slider = widgets.FloatSlider(description='Astig. angle', min=0, max=2 * np.pi, value=0, step=.1)
threefold_slider = widgets.FloatSlider(description='Threefold astig.', min=-2e3, max=1e3, value=0, step=10)
threefold_angle_slider = widgets.FloatSlider(description='Threefold astig. angle', min=0, max=2 * np.pi, value=0, step=.1)
coma_slider = widgets.FloatSlider(description='Coma', min=-2e3, max=2e3, value=0, step=10)
coma_angle_slider = widgets.FloatSlider(description='Coma angle', min=0, max=2 * np.pi, value=0, step=.1)

color_dropdown = widgets.Dropdown(description='Colormap',
                                  options=['Intensity', 'Complex', 'Amplitude', 'Phase'],
                                  value='Intensity')


def update(*args):
    probe.ctf.defocus = defocus_slider.value
    probe.ctf.Cs = cs_slider.value
    probe.ctf.C5 = c5_slider.value
    probe.ctf.astigmatism = astigmatism_slider.value
    probe.ctf.astigmatism_angle = astigmatism_angle_slider.value
    probe.ctf.C23 = threefold_slider.value
    probe.ctf.phi23 = threefold_angle_slider.value
    probe.ctf.coma = coma_slider.value
    probe.ctf.coma_angle = coma_angle_slider.value

    ctf_array = probe.ctf.evaluate_on_grid(probe.gpts[0], probe.extent[0] * 3 / 2)
    probe_array = np.fft.fftshift(np.fft.fft2(ctf_array))
    ctf_array = np.fft.fftshift(ctf_array)

    with artist1._mark.hold_sync():
        if color_dropdown.value == 'Intensity':
            artist1.image = domain_coloring(np.exp(1.j * np.angle(ctf_array)), pure_phase=True)
            intensity = np.abs(probe_array) ** 2
            artist2.image = np.tile((intensity / intensity.max())[..., None], (1, 1, 3))
        elif color_dropdown.value == 'Complex':
            artist1.image = domain_coloring(np.exp(1.j * np.angle(ctf_array)), pure_phase=True)
            #artist1.image = domain_coloring(ctf_array)
            artist2.image = domain_coloring(probe_array)
        elif color_dropdown.value == 'Amplitude':
            #artist1.image = domain_coloring(ctf_array)
            artist1.image = domain_coloring(np.exp(1.j * np.angle(ctf_array)), pure_phase=True)
            #artist1.image = np.tile((np.abs(ctf_array))[...,None],(1,1,3))
            intensity = np.abs(probe_array)
            artist2.image = np.tile((intensity / intensity.max())[..., None], (1, 1, 3))
        elif color_dropdown.value == 'Phase':
            artist1.image = domain_coloring(np.exp(1.j * np.angle(ctf_array)), pure_phase=True)
            artist2.image = domain_coloring(np.exp(1.j * np.angle(probe_array)), pure_phase=True)

        artist1.extent = [[-probe.cutoff_scattering_angles[0], probe.cutoff_scattering_angles[0]]] * 2


def reset_button_callback(*args):
    with artist1._mark.hold_sync(), artist2._mark.hold_sync():
        defocus_slider.value = 0
        cs_slider.value = 0
        c5_slider.value = 0
        astigmatism_slider.value = 0
        astigmatism_angle_slider.value = 0
        threefold_slider.value = 0
        threefold_angle_slider.value = 0
        coma_slider.value = 0
        coma_angle_slider.value = 0

toggle = widgets.ToggleButton(description='Use aliases', value=True)

def toggle_names(*args):
    if toggle.value:
        defocus_slider.description = 'Defocus'
        cs_slider.description = 'Cs'
        c5_slider.description = 'C5'
        astigmatism_slider.description = 'Astigmatism'
        astigmatism_angle_slider.description = 'Astig. angle'
        threefold_slider.description = 'Threefold astig.'
        threefold_angle_slider.description = 'Threefold astig. angle'
        coma_slider.description = 'Coma'
        coma_angle_slider.description = 'Coma angle'
    else:
        defocus_slider.description = '-C10'
        cs_slider.description = 'C30'
        c5_slider.description = 'C50'
        astigmatism_slider.description = 'C12'
        astigmatism_angle_slider.description = 'phi12'
        threefold_slider.description = 'C23'
        threefold_angle_slider.description = 'phi23'
        coma_slider.description = 'C21'
        coma_angle_slider.description = 'phi21'


toggle.observe(toggle_names)

update()

artist2.extent = [[0, probe.extent[0]], [0, probe.extent[1]]]

update = throttle(.1)(update)
defocus_slider.observe(update, 'value')
cs_slider.observe(update, 'value')
astigmatism_slider.observe(update, 'value')
astigmatism_angle_slider.observe(update, 'value')
threefold_slider.observe(update, 'value')
threefold_angle_slider.observe(update, 'value')
coma_slider.observe(update, 'value')
coma_angle_slider.observe(update, 'value')
c5_slider.observe(update, 'value')
color_dropdown.observe(update, 'value')

canvas1.x_label = 'Scattering angle x [mrad.]'
canvas1.y_label = 'Scattering angle y [mrad.]'
canvas1.title = 'Phase aberrations'

canvas2.x_label = 'x [Å]'
canvas2.y_label = 'y [Å]'
canvas2.title = 'Point spread function'

whitespace = widgets.HBox([])
whitespace.layout.height = '30px'

reset_button = widgets.Button(description='Reset')

canvas1.x_limits = (-60, 60)
canvas1.y_limits = (-60, 60)

reset_button.on_click(reset_button_callback)

widgets.HBox([canvas1.widget, canvas2.widget,
              widgets.VBox([whitespace,
                            color_dropdown,
                            defocus_slider,
                            cs_slider,
                            c5_slider,
                            astigmatism_slider,
                            astigmatism_angle_slider,
                            threefold_slider,
                            threefold_angle_slider,
                            coma_slider,
                            coma_angle_slider,
                            widgets.HBox([reset_button, toggle])])])